In [1]:
%load_ext autoreload
%autoreload 2
import os; import sys; sys.path.insert(0,'../')
import warnings
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

import preprocessing.spadl.api as spadl

In [2]:
## Configure file and folder names
datafolder = "../data"
statsbomb_json =  os.path.join(datafolder,"statsbomb-root","open-data-master","data")
statsbomb_h5 = os.path.join(datafolder,"statsbomb.h5")
spadl_h5 = os.path.join(datafolder,"spadl-statsbomb.h5")

### Convert raw Statsbomb json files to Statsbomb HDF5 file

In [3]:
spadl.statsbombjson_to_statsbombh5(statsbomb_json,statsbomb_h5)

...Adding competitions to ../data\statsbomb.h5
...Adding matches to ../data\statsbomb.h5
...Adding players and teams to ../data\statsbomb.h5


converting events files to ../data\statsbomb.h5: 100%|███████████████████████████████| 809/809 [02:38<00:00,  4.59it/s]


### Inspect StatsBomb HDF5 file

In [4]:
tablenames = ["matches","players","teams","competitions"]
tables = {name : pd.read_hdf(statsbomb_h5,key=name) for name in tablenames}

match_id = tables["matches"].match_id[0]
tables["events"] = pd.read_hdf(statsbomb_h5,f"events/match_{match_id}")

for k,df in tables.items():
    print("#",k)
    print(df.columns,"\n")

# matches
Index(['away_score', 'away_team_gender', 'away_team_group', 'away_team_id',
       'away_team_name', 'competition_id', 'competition_name', 'country_name',
       'data_version', 'home_score', 'home_team_gender', 'home_team_group',
       'home_team_id', 'home_team_name', 'id', 'kick_off', 'last_updated',
       'managers', 'match_date', 'match_id', 'match_status', 'match_week',
       'name', 'season_id', 'season_name', 'shot_fidelity_version',
       'xy_fidelity_version'],
      dtype='object') 

# players
Index(['country_id', 'country_name', 'extra', 'jersey_number', 'player_id',
       'player_name', 'player_nickname'],
      dtype='object') 

# teams
Index(['team_id', 'team_name'], dtype='object') 

# competitions
Index(['competition_gender', 'competition_id', 'competition_name',
       'country_name', 'match_available', 'match_updated', 'season_id',
       'season_name'],
      dtype='object') 

# events
Index(['counterpress', 'duration', 'extra', 'id', 'index', 'locati

### Convert Statsbomb data (in a HDF5 file) to the SPADL format (in a HDF5 file)

In [5]:
spadl.statsbombh5_to_spadlh5(statsbomb_h5,spadl_h5)

...Converting matches to games
...Converting players
...Converting teams
...Converting competitions
...Inserting actiontypes
...Inserting bodyparts
...Inserting results
... computing playergames (minutes played in each game


  0%|                                                                                        | 0/799 [00:00<?, ?game/s]


ValueError: DataFrame constructor not properly called!

### Inspect SPADL HDF5 file

In [ ]:
tablenames = ["games","players","teams","competitions","actiontypes","bodyparts","results"]
tables = {name : pd.read_hdf(spadl_h5,key=name) for name in tablenames}

game_id = tables["games"].game_id[0]
tables["actions"] = pd.read_hdf(spadl_h5,f"actions/game_{game_id}")

for k,df in tables.items():
    print("#",k)
    print(df.columns,"\n")

### (optional) Plotting actions
Extra library required: ```pip install matplotsoccer```

In [ ]:
import matplotsoccer

tablenames = [
    "games",
    "players",
    "teams",
    "competitions",
    "actiontypes",
    "bodyparts",
    "results",
]
tables = {name: pd.read_hdf(spadl_h5, key=name) for name in tablenames}

# Select England vs Belgium game at World Cup
games = tables["games"].merge(tables["competitions"])
game_id = games[(games.competition_name == "FIFA World Cup") 
              & (games.away_team_name == "England")
              & (games.home_team_name == "Belgium")].game_id.values[0]

actions = pd.read_hdf(spadl_h5, f"actions/game_{game_id}")
actions = (
    actions.merge(tables["actiontypes"])
    .merge(tables["results"])
    .merge(tables["bodyparts"])
    .merge(tables["players"],"left",on="player_id")
    .merge(tables["teams"],"left",on="team_id")
    .sort_values(["period_id", "time_seconds", "timestamp"])
    .reset_index(drop=True)
)

# use nickname if available else use full name
actions["player"] = actions[["player_nickname","player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)

#shot = 128
shot = 2201
a = actions[shot-4:shot+1]

games = tables["games"]
g = list(games[games.game_id == a.game_id.values[0]].itertuples())[0]
minute = int((a.period_id.values[0]-1)*45 +a.time_seconds.values[0] // 60) + 1
game_info = f"{g.match_date} {g.home_team_name} {g.home_score}-{g.away_score} {g.away_team_name} {minute}'"

print(game_info)

labels = a[["time_seconds", "type_name", "player", "team_name"]]
matplotsoccer.actions(
    location=a[["start_x", "start_y", "end_x", "end_y"]],
    action_type=a.type_name,
    team= a.team_name,
    result= a.result_name == "success",
    label=labels,
    labeltitle=["time","actiontype","player","team"],
    zoom=False,
    figsize=6
)
matplotsoccer.actions(
    location=a[["start_x", "start_y", "end_x", "end_y"]],
    action_type=a.type_name,
    team=a.team_name,
    result=a.result_name == "success",
    label=labels,
    labeltitle=["time","actiontype","player","team"],
    zoom=True,
)